# 3.1 Data transferred and ingested in butler and access to data


See if we can count images for some OR4 night  

In [1]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry

instrument = 'LSSTComCamSim'
repo = '/repo/LSSTComCam'  # Need to check the summit repo name.
# raw all collection contains all images 
collections=['LSSTComCamSim/raw/all']
butler = Butler(repo, collections=collections, instrument=instrument)
registry = butler.registry

print(butler.registry.dimensions["exposure"].RecordClass.fields)


exposure: 
  instrument: str
  id: int
  physical_filter: str
  obs_id: str
  exposure_time: float
  dark_time: float
  observation_type: str
  observation_reason: str
  day_obs: int
  seq_num: int
  seq_start: int
  seq_end: int
  group_name: str
  group_id: int
  target_name: str
  science_program: str
  tracking_ra: float
  tracking_dec: float
  sky_angle: float
  azimuth: float
  zenith_angle: float
  has_simulated: bool
  timespan: lsst.daf.butler.Timespan


Get raws for a given night we list the first and last detector and exposure id since the test asks for that. For LATISS the detector is always 0. 


In [2]:

dt='raw'
where="exposure.day_obs = 20240626"
datasetRefs = list(registry.queryDatasets(datasetType=dt,where=where))

print(f"There are {len(datasetRefs)} {dt} in collection: {collections} in: {repo} for {where}")

count = 0
for ref in datasetRefs[0], datasetRefs[-1]:
    count = count + 1
    print( ref.dataId['detector'], ref.dataId['exposure'])
    last_exposure = ref.dataId['exposure']

There are 599 raw in collection: ['LATISS/raw/all'] in: /repo/LATISS for exposure.day_obs = 20230511
0 2023051100001
0 2023051100599


Not needed for test but since we are here, let's get the last exposure.

In [5]:

dataId = {'exposure': last_exposure, 'detector': 0, }
img = butler.get('raw', **dataId)


Now we have 1 image
lets look at it  .. you do not need this following cell it is just in case you want to chekc the image.
Firefly will pop up in another tab.

In [4]:
import lsst.afw.display as afwDisplay

display = afwDisplay.getDisplay(backend='firefly')

display.mtv(img)



#3.2 LFA replication to USDF
Look in EFD for events with Large Files - check some of them

In [2]:
from lsst_efd_client import EfdClient, resample
loc = 'summit'

client = EfdClient(f'{loc}_efd')  
client.output = 'dataframe'
cl=client.influx_client
day = '2024-06-26'

query=f'''SELECT url, id FROM "lsst.sal.Electrometer.logevent_largeFileObjectAvailable" WHERE time >  '{day}' '''
result = await cl.query(query)


In [3]:
print(f"Got {len(result)} Large files availble since {day} at {loc}")

Got 173 Large files availble since 2023-04-20 at summit


#3.4 EFD data available
Choose at least 5 EFD topics at random from those available at the Summit. Choose at least 20 messages per topic from those available at the Summit, including 10 from the previous observing day and 10 from past history. Verify that all 100 messages are present in the USDF EFD.

In [23]:
import random
topics = await client.get_topics()

selected_topics = []
old_topics = []
results = {}
oldresults = {}

oldday = '2024-06-26'

# want to selct 5 topics randomly but with messages so randomize all indexes 
randoms = random.sample(range(0,len(topics)),len(topics))
for r in randoms:
   topic = topics[r]
   result = oldresult = []
   if len(selected_topics) < 4:
      query=f'''SELECT * FROM "{topic}" WHERE time >  '{day}T00:00:00.000Z' and time < '{day}T00:00:30.000Z' '''
      result = await cl.query(query)

   if len(old_topics) < 4:   
      query=f'''SELECT * FROM "{topic}" WHERE time >  '{oldday}T00:00:00.000Z' and time < '{oldday}T00:00:30.000Z' '''
      oldresult = await cl.query(query)

   if len(result) > 20:
        print (f"{topic} had {len(result)} messages")
        selected_topics.append(topic)
        results[topic] = result
        if len(selected_topics) > 4 and len(old_topics) > 4:
            break

   if len(oldresult) > 20:
        print (f"{topic} had {len(oldresult)} older messages")
        old_topics.append(topic)
        oldresults[topic] = oldresult
        if len(selected_topics) > 4 and len(old_topics) > 4:
            break


print(f"Random selction of five '{loc}' topics {selected_topics} with messages on {day}")
print(f"Random selction of five '{loc}' older topics {old_topics} with messages on {oldday}")

    

lsst.sal.ATPtg.mountPositions had 29 messages
lsst.sal.ATPtg.mountPositions had 29 older messages
lsst.sal.MTAirCompressor.analogData had 46 messages
lsst.sal.MTAirCompressor.analogData had 46 older messages
lsst.sal.MTMount.elevationDrives had 456 messages
lsst.sal.GIS.logevent_heartbeat had 30 messages
lsst.sal.GIS.logevent_heartbeat had 30 older messages
lsst.sal.ESS.lightningStrikeStatus had 30 older messages
Random selction of five 'summit' topics ['lsst.sal.ATPtg.mountPositions', 'lsst.sal.MTAirCompressor.analogData', 'lsst.sal.MTMount.elevationDrives', 'lsst.sal.GIS.logevent_heartbeat'] with messages on 2023-04-20
Random selction of five 'summit' older topics ['lsst.sal.ATPtg.mountPositions', 'lsst.sal.MTAirCompressor.analogData', 'lsst.sal.GIS.logevent_heartbeat', 'lsst.sal.ESS.lightningStrikeStatus'] with messages on 2023-05-02


In [32]:
import pickle
with open('test_data.pickle', 'wb') as file:
    pickle.dump(results,file, protocol=pickle.HIGHEST_PROTOCOL)

with open('old_test_data.pickle', 'wb') as file:
     pickle.dump(oldresults,file, protocol=pickle.HIGHEST_PROTOCOL)

#move these files to USDF
